In [ ]:
%env OGDF_BUILD_DIR=~/Bachelorarbeit/OGDF/build-debug

from ogdf_python_widget import ogdf, cppinclude
from ogdf_python_widget.widget import Widget
import ogdf_python_widget

cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")

G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.randomPlanarTriconnectedGraph(G, 20, 40)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)

for n in G.nodes:
    GA.label[n] = "N%s" % n.index()

SL = ogdf.SugiyamaLayout()
SL.call(GA)

w = Widget(GA)

w.on_link_click_callback = lambda link, alt, ctrl: clickLink(link, alt, ctrl)
w.on_node_click_callback = lambda node, alt, ctrl: clickNode(node, alt, ctrl)
w.on_svg_click_callback = lambda x, y, alt, ctrl, backgroundClicked: clickSvg(x, y, alt, ctrl, backgroundClicked)

movingLinkIds = []
selectedNode = None

def clickNode(node, alt, ctrl):
    global w, selectedNode, G, GA
    if alt:
        #deletes node when clicking on it while holding alt key
        G.delNode(node)
    if ctrl:
        #adds links when holding ctrl
        if selectedNode == None:
            GA.fillColor[node] = ogdf.Color(150, 231, 255)
            w.update_node(node)
            selectedNode = node
        else:
            G.newEdge(selectedNode, node)
            GA.fillColor[selectedNode] = ogdf.Color(255, 255, 255)
            w.update_node(selectedNode)
            selectedNode = None
    
def clickLink(link, alt, ctrl):
    global w, movingLinkIds, G
    if alt:
        #deletes link when clicking on it while holding alt key
        G.delEdge(link)
    if ctrl:
        #ctrl toggles link movement
        if link.index() in movingLinkIds:
            w.remove_bend_mover_for_id(link.index())
            movingLinkIds.remove(link.index())
        else:
            w.move_link(link)
            movingLinkIds.append(link.index())
        
def clickSvg(x, y, alt, ctrl, backgroundClicked):
    global w, G, GA
    if backgroundClicked and ctrl:
        #adds node when clicking svg and holding ctrl
        coords = w.viewcoords_to_svgcoords(x,y)
        n = G.newNode()
        GA.x[n] = coords['x']
        GA.y[n] = coords['y']
        GA.label[n] = "N%s" % n.index()
        w.update_node(n)


import ipywidgets as widgets
from ipywidgets import HBox, VBox

##################################################################################

addNodeMoveToggle = widgets.Checkbox(
    value=False,
    description='move Nodes',
    disabled=False,
    indent=False
)

def on_addNodeMoveToggle(change):
    global w, addNodeMoveToggle
    w.enable_node_movement(addNodeMoveToggle.value)
addNodeMoveToggle.observe(on_addNodeMoveToggle, 'value')

################################################################################## 

hbox = HBox([addNodeMoveToggle])
vbox = VBox([hbox, w])

vbox

In [ ]:
w.click_thickness = 8

In [ ]:
n = G.newNode()
GA.label[n] = "N%s" % n.index()
w.update_node(n)
e = G.newEdge(G.nodes[2], n)
e2 = G.newEdge(G.nodes[1], n)

In [ ]:
GA.x[n] = -1342
GA.y[n] = -1311
for edge in n.adjEntries:
    w.update_link(edge.theEdge())
w.update_node(n)

In [ ]:
G.delNode(G.nodes.begin().succ())

In [ ]:
w.send({'code': 'test'})
w.zoom = 1
w.x_pos = 0
w.y_pos = 0

In [ ]:
w.enable_bend_movement(True)
w.enable_node_movement(True)

In [ ]:
w.export_graph()

In [ ]:
w.update_node(n)